In [324]:
def remove_redis_db():    
    rds = redis.StrictRedis(host='docker.vm', port=6379, db=15)
    for key in rds.scan_iter("*"):
        rds.delete(key)
#remove_redis_db()

In [282]:
import redis
def get_raw_data():
    rds = redis.StrictRedis(host='docker.vm', port=6379, db=14)
    for key in rds.scan_iter("data_raw:*"):
        Accelerations=rds.get(key).decode('ascii').split(",")        
        Accelerations=[int(v) for v in Accelerations]
        
        assert len(Accelerations) %3 ==0, f"data corrupt with key {key}"
        
        all_accelerometer,all_time=chop_data_to_slice(Accelerations)
        for a_chunk,time_chunk in zip(all_accelerometer,all_time):
            yield a_chunk,time_chunk            
    yield None,None

In [40]:
#1. 把加速度数据切割成不同的段
#2. 时间重整化，t0时刻设置为0
#chop into data slice
def chop_data_to_slice(a_):
    all_accelerometer,all_time=[],[]
    while len(a_)>0:
        start_time,end_time,length=a_[0],a_[1],a_[2]
        data_grids=(length-3)/3
        assert start_time>365*86400*1000,"start_time corrupt"
        assert end_time>365*86400*1000,"end_time corrupt"
        all_accelerometer.append(a_[3:length])
        all_time.append((start_time,(end_time-start_time)/data_grids))
        a_=a_[length:]
    lens=[len(v) for v in all_accelerometer]
    print(f"😊 data successfully chopped into {len(all_accelerometer)} pieces ,average length {sum(lens)/len(lens)}, min {min(lens)}, max {max(lens)}")
    return all_accelerometer,all_time


In [316]:
#从原始信号中选出最能符合最多平滑周期约束的信号
def shot(peaks_candidate,peak_index,i):
    global DEBUG 
    t0=peak_index[i]
    period=t0-peak_index[i-1]
    assert period>1.0,"bad period"
    shotted=[t0]
    shotted_possible=[t0]
    time_drift=0.15*period
    #左向匹配
    cur=t0-period
    #if DEBUG :       print(f"cur {cur} period {period}")
    while cur>=0:
        #放这里而不是最后，避免下标越界
        shotted_possible.append(int(cur))
        
        ib=max(0,int(cur-time_drift))
        ie=min(len(peaks_candidate),int(cur+time_drift))
        ind=np.argmax(peaks_candidate[ib:ie])
        #print(f"to left:ib {ib},ie {ie}, period {period},i {i} cur {cur} ind{ind}")
        if peaks_candidate[ib+ind]>0:
            cur=ib+ind #接收现实的修正
            shotted.append(int(cur))
            shotted_possible[-1]=cur
        cur-=period
            
    #向右匹配
    cur=t0+period
    while cur<len(peaks_candidate):
        #放这里而不是最后，避免下标越界
        shotted_possible.append(int(cur))
        
        ib=max(0,int(cur-time_drift))
        ie=min(len(peaks_candidate),int(cur+time_drift))
        ind=np.argmax(peaks_candidate[ib:ie])
        #print(f"to right:ib {ib},ie {ie}, period {period},i {i} cur {cur} ind{ind}")
        if peaks_candidate[ib+ind]>0:
            cur=ib+ind #接收现实的修正
            shotted.append(int(cur))
            shotted_possible[-1]=cur
        cur+=period
    return shotted,shotted_possible
def best_periodical(peaks_candidate):
    peak_index=np.nonzero(peaks_candidate)[0]
    assert len(peak_index)>10, "peaks_candidate contains too less data points"
    
    #假设任意两个峰值信号，作为心跳信号。分析这个假设下出现的心跳周期数
    #every element contains tuple,which is (shotted,shotted_missed)
    shoted_list=[shot(peaks_candidate,peak_index,i) for i in range(1,len(peak_index))]
    duty_cycles=[ len(s[0])/len(peak_index)  for s in shoted_list]
    
    #print(f"duty_cycles {duty_cycles} ")
    #找出最大的心跳周期
    maxind=np.argmax(np.array(duty_cycles))
    #print(f"duty_cycles len{len(duty_cycles)} maxind {maxind} max timespan {time_span[maxind]} duty_cycle {duty_cycles[maxind]} t0 {duty_cycles_states[maxind].t0}")
    
    #把数据写回到时间序列
    result=[0]*len(peaks_candidate)
    for i in shoted_list[maxind][0]:
        result[i]=peaks_candidate[i]
        
    possible=[0]*len(peaks_candidate)
    maxv,l=np.max(peaks_candidate),len(shoted_list[maxind][1])
    #print("shoted_list[maxind][1]",shoted_list[maxind][1],f"len of peaks_candidate {len(peaks_candidate)}")
    for i,vi in enumerate(shoted_list[maxind][1]):
        #possible[vi]=peaks_candidate[vi]
        possible[vi]=maxv*0.7*(i)/l +(maxv*0.3)
    #print(f"possible length {np.count_nonzero(possible)}")
    return result,possible

In [319]:
def pickout_peak(data,grids_span):
    value=[0]*len(data)
    for i,v in enumerate(data):
        l,r=max(0,int(i-grids_span)),min(int(i+grids_span),len(data))
        assert r-l>1,"can not peak out signal peak"
        j=np.argmax(data[l:r])
        if j==i-l:
            value[i]=v
    return value

In [323]:
import json
import math
DEBUG=False
# 60*1000/220=272,272,60*1000/39=1538,1538  
scan_iteration_foreach_window,time_window_min,time_window_max = 5,(60*1000/220)*0.9,(60*1000/39)*0.9
# x^scan_iteration_foreach_window=time_window_max/time_window_min,ig. 1538/272
factor=math.exp(math.log(time_window_max/time_window_min)/(scan_iteration_foreach_window-1))
time_windows=[round(time_window_min*math.pow(factor,i)) for i in range(scan_iteration_foreach_window)]
def labeling_heart_beat(accelerations,time_info):  
    global DEBUG 
    startTime,timespan=time_info[0],time_info[1]
    
    #密度太稀疏，一秒钟40个数据点以上的不要
    if timespan>25:
        return
    # if not startTime in  [1651932984341,1651933735094,1651934155535,1651934756171,1651955481794,1651955782167,1651955962357,1651935446909]:
    #     return
    if rds_to.exists(str(startTime)): return True    
    
    x,y,z=accelerations[0::3],accelerations[1::3],accelerations[2::3] #axis is x or y or z
    assert len(accelerations)%3==0 and len(x)== len(y) and len(y)==len(z),"data corrupt! check pls"    
    a=[math.sqrt(v**2+y[i]**2+z[i]**2) for i,v in enumerate(x)  ]
    
    best_score=-1
    Periodical=[]
    bestAxis=[]
    channelI=-1
    bestSpans=-1       
    
    for span in time_windows: 
        for channel,axis in enumerate([x,y,z,a]): #axis is x or y or z  
            #DEBUG= (channel == 2) and (span == 583)
    
            absaxis=np.array([abs(v) for v in axis],dtype="float32")
            #print(f"absaxis {absaxis.tolist()} len {len(absaxis)} span {span} timespan{timespan} ")
            peaks_candidate=pickout_peak(absaxis,span/timespan)
            num_peak=np.count_nonzero(peaks_candidate)
            if num_peak<15: continue
            peaks_periodical,peak_possible=best_periodical(peaks_candidate)
            num_periodical=np.count_nonzero(peaks_periodical)
            num_possible=np.count_nonzero(peak_possible)
            #print(f"peaks_periodical num {num_periodical} from num_peak {num_peak} num_possible {num_possible}")
            if num_periodical<10: continue
            #要求num_possible 和 num_periodical 尽可能一致，做log 惩罚
            #print(f"num_possible/num_periodical {num_possible} {num_periodical} {num_possible/num_periodical}")
            score=num_periodical/(np.log(num_possible/num_periodical)+0.001)
            #score=num_periodical/np.exp(num_possible/num_periodical)
#             #插值补充点
            #believable_added=add_believable_data(peaks_periodical,axis,time)            
            
            #得分100以下可能不准确，不保留结果
            if score<100:
                continue
            if score>=best_score:
                best_score=score
                Periodical=peak_possible
                channelI=channel
                bestAxis=axis
                bestSpans=span   
    if best_score>0:
        time=[timespan*i for i in range(len(bestAxis))]
        label= [1 if v!=0 else 0 for v in Periodical]
        
        rds_to.set(str(startTime), json.dumps({"accelero":accelerations,"start":startTime,"timespan":timespan,"label":label}))        
        print(f"saved to redis ,key:{startTime}",{best_score})
       
        # plt.figure(startTime,figsize=(30, 6))
        # plt.xlabel(f"data {startTime} ,channel ${channelI}"+f",bestSpans ${bestSpans}"+f",best_score{best_score} ")
        # plt.plot(time,bestAxis)
        # plt.plot(time,Periodical)
    
rds_to = redis.StrictRedis(host='docker.vm', port=6379, db=15)
data_source=get_raw_data() 
cnt=0
while True:
    accelerations,time_span= next(data_source)   
    if accelerations == None or time_span==None:break  
    # cnt+=1
    # if cnt%37!=0:
    #     continue
    labeling_heart_beat(accelerations,time_span) 
print("Job completed!")

😊 data successfully chopped into 258 pieces ,average length 5399.186046511628, min 4101, max 5406
saved to redis ,key:1651836110713 {37000.0}
saved to redis ,key:1651836140835 {36000.0}
saved to redis ,key:1651836170860 {34000.0}
saved to redis ,key:1651836200904 {34000.0}
saved to redis ,key:1651836230924 {35000.0}
saved to redis ,key:1651836260956 {1133.8043582474686}
saved to redis ,key:1651836290988 {35000.0}
saved to redis ,key:1651843837355 {34000.0}
saved to redis ,key:1651843867478 {34000.0}
saved to redis ,key:1651843938536 {237.78735451237762}
saved to redis ,key:1651843968657 {32000.0}
saved to redis ,key:1651843998683 {32000.0}
saved to redis ,key:1651844028729 {34000.0}
saved to redis ,key:1651844058748 {946.6025092728122}
saved to redis ,key:1651844088779 {32000.0}
saved to redis ,key:1651844118795 {34000.0}
saved to redis ,key:1651844148827 {32000.0}
saved to redis ,key:1651844178859 {146.98344757121916}
saved to redis ,key:1651844208902 {31000.0}
saved to redis ,key:165